part 3 is for exploratory data analysis (EDA) of classes which have less scores than other respective classes & adjusting these classes' images respectively

# Imports

In [80]:
from numpy.random import seed
seed(42)
import tensorflow as tf
tf.random.set_seed(42)

import pandas as pd
import numpy as np

import sys
import string
import logging
import regex as re
import pickle
import os
import glob
import base64

from IPython.display import display
import matplotlib.pyplot  as plt
from PIL import Image, ImageFile, ImageOps, ImageDraw
from pathlib import Path

import cufflinks as cf
import chart_studio.plotly as py
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
import plotly.io as pio
from plotly.offline import init_notebook_mode

from tqdm.notebook import tqdm
from collections import Counter
from keras_preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from concurrent.futures import ThreadPoolExecutor, as_completed, wait
from functools import partial

import fastdup

# Global Variables & Functions

In [10]:
dataDir = 'dataset/'
dataReDir = 'dataset_related/'
imgTypes = list(filter(lambda v: re.match('\d{1,3}\. ', v), os.listdir(dataDir)))
imgTypes

['00. selfies',
 '10. fmemes',
 '20. ememes',
 '30. fSocialMedia',
 '40. eSocialMedia',
 '50. fTxtMssgs',
 '70. eGreetingAndMisc',
 '81. academicPhotos',
 '82. academicDigital']

In [11]:
def pklSave(contentToBeSaved, fullPath):
    with open(fullPath, 'wb') as f:
        pickle.dump(contentToBeSaved, f)

def pklLoad(fullPath):
    with open(fullPath, 'rb') as f:
        content = pickle.load(f)
    return content

def pklForceLoad(path, dtype = 'dict'):
    try:
        content = pklLoad(path)
        return content
    except Exception as e:
        if dtype == 'list':
            pklSave([], path)
            return []
        else:
            pklSave({}, path)
            return {}

# more about naming standards for path components here: https://stackoverflow.com/questions/2235173/what-is-the-naming-standard-for-path-components
def joinPaths(baseDirectory, relativePath):
    return os.path.normpath(os.path.join(baseDirectory, relativePath))

def changeSep(path, newSep, oldSep=os.path.sep): 
    # source: https://stackoverflow.com/questions/18707338/print-raw-string-from-variable-not-getting-the-answers#:~:text=To%20turn%20a%20variable%20to%20raw%20str%2C%20just%20use
    return os.path.normpath(rf"{path}").replace(os.path.sep, newSep) # source: https://mail.python.org/pipermail/tutor/2011-July/084788.html

def pillowOpenOriented(path):
    img = Image.open(path)
    return ImageOps.exif_transpose(img)

## Setting up images/Labels' dictionaries and the metadata csv file

In [12]:
imgTypeToNum = pklLoad('./dataset_related./imgTypeToNum.pickle')
numToImgType = {}
for  i, type in enumerate(imgTypes):
    imgTypeToNum[type] = i
    numToImgType[i] = type
imgTypeToNum

{'00. selfies': 0,
 '10. fmemes': 1,
 '20. ememes': 2,
 '30. fSocialMedia': 3,
 '40. eSocialMedia': 4,
 '50. fTxtMssgs': 5,
 '70. eGreetingAndMisc': 6,
 '81. academicPhotos': 7,
 '82. academicDigital': 8}

In [13]:
imgs_metadata_v2 = pd.read_csv('./dataset_related/imgsPropsv2.csv')

def combine_paths(row):
    return joinPaths(row['relative_path'], row['file_name'])
imgs_metadata_v2['full_path'] = imgs_metadata_v2.apply(combine_paths, axis=1)
imgs_metadata_v2.set_index('full_path', inplace=True)

imgs_metadata_v2.sample(5)

# side note: the "mixed types" warnings are for 
# Index(['ar_words_original', 'ar_words_lemmatized', 'ar_words_including_separators'], dtype='object')
# columns, which currently don't interest us, so we'll neglect this warning for now.

C:\Users\ashra\AppData\Local\Temp\ipykernel_26220\374723971.py:1: DtypeWarning:

Columns (25,28,30) have mixed types. Specify dtype option on import or set low_memory=False.



,file_name,relative_path,aspect_ratio,area,width,height,dominant_color_1,color_to_image_ratio_1,dominant_color_2,color_to_image_ratio_2,...,en_words_segmented_and_lemmatized,ar_words_lemmatized,en_words_including_separators,ar_words_including_separators,en_words_to_boxes,ar_words_to_boxes,en_avg_score,ar_avg_score,math_digits_or_symbols_count,class
full_path,,,,,,,,,,,,,,,,,,,,,
dataset\40. eSocialMedia\eSocialMedia0000875.jpg,eSocialMedia0000875.jpg,dataset/40. eSocialMedia,2.31,297252,828,359,16777215,0.70621,16711422,0.05507,...,muhammed usseirdtnooxgonboysert,اتجوزيني وهبقى أزمة انا معنديش أزمة هعملك وهبق...,Muhammed usseir || 2d ||| TnooX || Gonboysert,اتجوزيني وهبقى وقت الأزمات || بس انا معنديش أز...,"{'Muhammed': [[[123.0, 62.0], [286.0, 62.0], [...","{'الأزمات': [[[117.0, 147.0], [243.0, 142.0], ...",0.72223,0.98994,0,40. eSocialMedia
dataset\10. fmemes\fmemes0010267.jpg,fmemes0010267.jpg,dataset/10. fmemes,1.00,250000,500,500,16777215,0.04882,0,0.02790,...,ira lpa belloni de ouemadosreirtedeellosne gen...,NaN,IR ALPABELLONIDEOUEMADOS ||| REIRTEDEELLOS || ...,NaN,"{'IR ALPABELLONIDEOUEMADOS': [[[17.0, 11.0], [...",{},0.95616,0.00000,0,10. fmemes
dataset\10. fmemes\fmemes0009228.jpg,fmemes0009228.jpg,dataset/10. fmemes,1.50,166500,500,333,16777215,0.06508,0,0.03053,...,dotvneoneoesehsiamasloha gas gratis,NaN,DOTVNEONEOESEHS ||| IAMAS LO HAGAS GRATIS,NaN,"{'DOTVNEONEOESEHS': [[[42.0, 11.0], [457.0, 13...",{},0.79763,0.00000,0,10. fmemes
dataset\00. selfies\selfies0022023.jpg,selfies0022023.jpg,dataset/00. selfies,1.00,93636,306,306,16777215,0.37274,16711679,0.00578,...,NaN,NaN,NaN,NaN,{},{},0.00000,0.00000,0,00. selfies
dataset\70. eGreetingAndMisc\eGreetingAndMisc0006165.jpg,eGreetingAndMisc0006165.jpg,dataset/70. eGreetingAndMisc,2.10,50530,326,155,3876,0.35650,3878,0.00671,...,NaN,مبار محتوى عيدكه عام خير,NaN,مبارك محتويات || عيدكه || وكل عام وأنتم بخير,{},"{'محتويات': [[[281.0, 2.0], [318.0, 2.0], [318...",0.00000,0.98155,0,70. eGreetingAndMisc


# Further Refinement of Dataset

As we can see from the best model scores table in `comparing_models_results.ipynb` (before introducing models v3.x):

<img src='./project_media/best_scores_table_before_v3.png' width=400 />

the two most classes that we should aim to improve are `academicDigital` and `fSocialMedia` classes
others we could try and improve as well are `ememes` and `academicPhotos`

## Functions to Measure & Order Images 

In [14]:
def get_image_ratio(img_or_path):
    if isinstance(img_or_path, str):
        img = Image.open(img_or_path)
    else:
        img = img_or_path
    width, height = img.size
    ratio = round(float(width) / float(height), 3)
    return ratio

# draft: use this only if you don't have a csv containing img's metadata 
# (which in my case, got created in dataset_preprocessing_part_2.ipynb)
def get_text_boxes_num(img_or_path):
    if isinstance(img_or_path, str):
        img = Image.open(img_or_path)
    else:
        img = img_or_path

    # import check
    if 'paddleocr' in sys.modules:
        from paddleocr import PaddleOCR
        paddle_ocr = PaddleOCR(use_angle_cls=True, use_gpu=True, lang="en")
        paddle_ocr = PaddleOCR(use_angle_cls=True, use_gpu=True, lang="ar")

    try:
        return len(paddle_ocr.ocr(np.array(img), det=True, rec=False, cls=True)[0])
    except:
        # then '[0]' didn't work as no boxes were found, so return 0
        return 0

def num_detected_words(img_path):
    global imgs_metadata_v2
    return imgs_metadata_v2.loc[img_path, 'total_en_boxes'] + imgs_metadata_v2.loc[img_path, 'total_ar_boxes']

## Functions to Manage Images' Removal

In [148]:
from typing import Callable, Union

def _format_num(x):
    return format(x, '07d')

def prefix_imgs_by_score(dir:str, scoring_func:Callable[[Union[str, Image.Image]], float], asc=False):
    '''
    in images inside `dir`, their names will be like:
    0_img004.jpg, 1_img007.jpg, 2_img002.jpg, ...
    of course, the `img004`/etc part is not exact, but depends on your original 
    naming structure of the images 
    '''
    # Get a list of image filenames in the directory
    filenames = sorted(os.listdir(dir))
    # Calculate scores for each image using the scoring function
    scores = [(filename, scoring_func(joinPaths(dir, filename))) for filename in filenames]

    # Sort the filenames based on the scores in descending order
    sorted_old_filenames_with_scores = sorted(scores, key=lambda x: x[1], reverse=(not asc))
    # side note: used _format_num() instead of directly using "i" to maintain structure 
    # (helps when we want to sort list by string)
    sorted_filenames = [f"{_format_num(i)}_{filename}" for i, (filename, _) in enumerate(sorted_old_filenames_with_scores)]

    # map each prefix number (e.g., '0' in '0_', etc) to filename
    # to be possibly used later in rename_imgs()
    prefix_to_filename_with_prefix = {i : filename for i, filename in enumerate(sorted_filenames)}

    # Rename the files with the sorted filenames
    for (old_filename, _), new_filename in zip(sorted_old_filenames_with_scores, sorted_filenames):
        old_path = joinPaths(dir, old_filename)
        new_path = joinPaths(dir, new_filename)
        os.rename(old_path, new_path)
    
    return prefix_to_filename_with_prefix

def _alter_orig_filename_func(filename:str, id:int):
    '''
    Example:
    'asdf/dfklje/eSocialMedia0000007.jpg'
    will be:
    'asdf/dfklje/eSocialMedia0000100.jpg'
    if id = 100, and so on...
    '''
    return re.sub(r'(\d+)(?=\.[^.]+$)', _format_num(id), filename)

def removing_imgs_prefix(dir:str, old_prefix_to_filename:dict, alter_orig_filename=False):
    '''
    this function should be called after you manually/automatically 
    remove the unwanted images from `dir`. it does the following:
    stores the removed filenames (without the prefix, i.e., "0_") and returns that list of names
    Furthermore, in images inside `dir`, the filenames will change from:
    0_img004.jpg, 1_img007.jpg, 2_img002.jpg, ...
    to this if alter_orig_filename=False:
    img004.jpg,   img007.jpg,    img002.jpg, ... IF 
    else:
    img001.jpg,   img002.jpg,    img003.jpg, ... IF alter_orig_filename=True

    of course, the `img004`/etc part is not exact, but depends on your original 
    naming structure of the images
    '''
    # Get a list of image filenames in the directory
    # and the final prefix value that was set by prefix_imgs_by_score() 
    # which should be equal to the number of images in dir - 1 (as 0-based prefix is used)
    filenames = sorted(os.listdir(dir))
    final_prefix = max(old_prefix_to_filename.keys())
    
    old_paths, new_paths = [], []
    removed_filenames = []
    
    # contains new prefix/filename pairs (after removing unwanted images from `dir`)
    updated_prefix_to_filename = {int(filename_with_prefix.split('_', 1)[0]) : filename_with_prefix.split('_', 1)[1]
                                  for filename_with_prefix in filenames}
    
    # `old_prefix_to_filename` contains all prefix/filename pairs (even those that you removed from `dir`)
    for old_prefix in old_prefix_to_filename.keys():
        filename = updated_prefix_to_filename.get(old_prefix)
        if filename is None:
            removed_filenames.append(old_prefix_to_filename[old_prefix])
        else:
            old_paths.append(joinPaths(dir, _format_num(old_prefix) + '_' + filename))
            new_paths.append(joinPaths(dir, filename))

    # removing prefixes from filenames in `dir`
    for i in range(len(new_paths)):
        if alter_orig_filename:
            # changing id of filename to be sequential (check below function's docstring for example)
            new_paths[i] = _alter_orig_filename_func(new_paths[i], i)
        os.rename(old_paths[i], new_paths[i])
    
    return removed_filenames

def update_metadata_df(df, removed_filenames, have_prefix=True, full_path=False, alter_orig_filename=False) -> pd.DataFrame:
    if full_path:
        removed_filenames = [os.path.basename(fn) for fn in removed_filenames]
    if have_prefix:
        removed_filenames_without_prefix = [fn.split('_', 1)[1] for fn in removed_filenames]
    else:
        removed_filenames_without_prefix = removed_filenames
    # Removing rows based on condition on column 'filename'
    condition = ~df['file_name'].isin(removed_filenames_without_prefix)
    df = df[condition]
    if alter_orig_filename:
        # TODO later: adjust df if filenames (including id) have been altered (out of current scope)
        pass
    return df 

def remove_imgs(filepaths_to_remove:list[str]):
    for fp in filepaths_to_remove:
        os.remove(fp)
    return filepaths_to_remove

## Filtering `academicDigital` Dataset

### By Ratio

In [56]:
class_path = joinPaths(dataDir, '82. academicDigital/')

prefix_to_filename_with_prefix = prefix_imgs_by_score(class_path, get_image_ratio, asc=False)

In [64]:
# removed_filenames are the unwanted images that you manually removed from `dir`
removed_filenames = removing_imgs_prefix(class_path, prefix_to_filename_with_prefix)
removed_filenames

['0000000_academicDigital0000045.jpg',
 '0000001_academicDigital0000416.jpg',
 '0000002_academicDigital0001023.jpg',
 '0000003_academicDigital0000641.jpg',
 '0000004_academicDigital0000071.jpg',
 '0000005_academicDigital0000073.jpg',
 '0000006_academicDigital0000082.jpg',
 '0000007_academicDigital0000106.jpg',
 '0000008_academicDigital0000081.jpg',
 '0000009_academicDigital0000084.jpg',
 '0000010_academicDigital0000673.jpg',
 '0000011_academicDigital0000367.jpg',
 '0000012_academicDigital0000368.jpg',
 '0000013_academicDigital0000143.jpg',
 '0000014_academicDigital0000046.jpg',
 '0000015_academicDigital0000792.jpg',
 '0000016_academicDigital0001010.jpg',
 '0000017_academicDigital0001516.jpg',
 '0000018_academicDigital0001358.jpg',
 '0000019_academicDigital0000313.jpg',
 '0000020_academicDigital0000154.jpg',
 '0000021_academicDigital0001196.jpg',
 '0000022_academicDigital0000720.jpg',
 '0000023_academicDigital0001586.jpg',
 '0000024_academicDigital0000356.jpg',
 '0000025_academicDigital

In [76]:
imgs_metadata_v3_ratio = update_metadata_df(imgs_metadata_v2, removed_filenames)
imgs_metadata_v3_ratio[imgs_metadata_v3_ratio['class'] == '82. academicDigital'].shape

(1626, 37)

Before: 1979 --> after: 1626

sample of images with high ratio removed (total: 293):

<img src='./project_media/academicDigital high ratio.png' width=1100 />


sample of images with low ratio removed (total: 60):

<img src='./project_media/academicDigital low ratio.png' width=1100 />

### By #words

In [79]:
prefix_to_filename_with_prefix = prefix_imgs_by_score(class_path, num_detected_words, asc=False)

In [82]:
# removed_filenames are the unwanted images that you manually removed from `dir`
removed_filenames = removing_imgs_prefix(class_path, prefix_to_filename_with_prefix)
removed_filenames

['0000729_academicDigital0001444.jpg',
 '0000883_academicDigital0001478.jpg',
 '0000925_academicDigital0000582.jpg',
 '0000927_academicDigital0000978.jpg',
 '0000947_academicDigital0000583.jpg',
 '0000959_academicDigital0000469.jpg',
 '0000979_academicDigital0000563.jpg',
 '0001013_academicDigital0001107.jpg',
 '0001030_academicDigital0001060.jpg',
 '0001044_academicDigital0000809.jpg',
 '0001179_academicDigital0001325.jpg',
 '0001186_academicDigital0000670.jpg',
 '0001332_academicDigital0000472.jpg',
 '0001344_academicDigital0000980.jpg',
 '0001345_academicDigital0001106.jpg',
 '0001423_academicDigital0000119.jpg',
 '0001424_academicDigital0000744.jpg',
 '0001425_academicDigital0000868.jpg',
 '0001427_academicDigital0000910.jpg',
 '0001428_academicDigital0000913.jpg',
 '0001429_academicDigital0000991.jpg',
 '0001434_academicDigital0000095.jpg',
 '0001437_academicDigital0000537.jpg',
 '0001439_academicDigital0000912.jpg',
 '0001442_academicDigital0001189.jpg',
 '0001446_academicDigital

In [84]:
imgs_metadata_v3_ratio_texts = update_metadata_df(imgs_metadata_v3_ratio, removed_filenames)
imgs_metadata_v3_ratio_texts[imgs_metadata_v3_ratio_texts['class'] == '82. academicDigital'].shape

(1422, 37)

Before: 1626 --> after: 1422

sample of images with few words (total: 204):

<img src='./project_media/academicDigital few words.png' width=1100 />


### By `fastdup` Functions

In [135]:
work_dir = './dataset_related/fastdup_work_dir_academicDigital'
input_dir = joinPaths(dataDir, '82. academicDigital/')

In [137]:
fd = fastdup.create(work_dir, input_dir)
fd.run()

FastDup Software, (C) copyright 2022 Dr. Amir Alush and Dr. Danny Bickson.

 ########################################################################################

Dataset Analysis Summary: 

    Dataset contains 1422 images
    Valid images are 100.00% (1,422) of the data, invalid are 0.00% (0) of the data
    Similarity:  20.82% (296) belong to 19 similarity clusters (components).
    79.18% (1,126) images do not belong to any similarity cluster.
    Largest cluster has 554 (38.96%) images.
    For a detailed analysis, use `.connected_components()`
(similarity threshold used is 0.9, connected component threshold used is 0.96).

    Outliers: 5.84% (83) of images are possible outliers, and fall in the bottom 5.00% of similarity values.
    For a detailed list of outliers, use `.outliers()`.


In [139]:
fastdup.run_kmeans(input_dir, work_dir, num_clusters=10, nearest_neighbors_k=1, high_accuracy=True)

FastDup Software, (C) copyright 2022 Dr. Amir Alush and Dr. Danny Bickson.


0

In [140]:
fastdup.create_kmeans_clusters_gallery(work_dir, save_path='./dataset_related/fastdup_academicDigital_kmeans')

100%|██████████| 10/10 [00:07<00:00,  1.34it/s]


Finished OK. Components are stored as image files ./dataset_related/fastdup_academicDigital_kmeans/components_[index].jpg
Stored kmeans clusters visual view in  ./dataset_related/fastdup_academicDigital_kmeans\components.html
Execution time in seconds 8.9


0

In [141]:
kmeans_df = pd.read_csv(joinPaths(work_dir, 'kmeans_assignments.csv'))

In [144]:
kmeans_cluster_filenames = kmeans_df.groupby('cluster').agg({'filename': list, 'distance': max})
cluster_to_filenames = {}
for cluster_idx in kmeans_cluster_filenames.index:
    cluster_to_filenames[cluster_idx] = kmeans_cluster_filenames.loc[cluster_idx, 'filename']

total = 0
for cluster_idx, filenames in cluster_to_filenames.items():
    print(cluster_idx, len(filenames))
    total += len(filenames)
print(total)

0 128
1 234
2 222
3 90
4 113
5 252
6 48
7 87
8 242
9 6
1422


Before: 1422 --> after: 1368

Checking out cluster 6 and 9's images from `___/components.html`, 
we see that these images are actually taken by mobile camera, not digitally, so they'll be removed:

<img src='./project_media/academicDigital should be academicPhotos.png' width=1000 />

In [151]:
filepaths_to_remove = cluster_to_filenames[6] + cluster_to_filenames[9]
len(filepaths_to_remove)

54

In [152]:
removed_filepaths = remove_imgs(filepaths_to_remove)

In [153]:
imgs_metadata_v3_acdig = update_metadata_df(imgs_metadata_v3_ratio_texts, removed_filepaths, have_prefix=False, full_path=True)
imgs_metadata_v3_acdig[imgs_metadata_v3_acdig['class'] == '82. academicDigital'].shape

(1368, 37)

## Filtering `fSocialMedia` Dataset

### By Ratio

In [158]:
class_path = joinPaths(dataDir, '30. fSocialMedia/')

prefix_to_filename_with_prefix = prefix_imgs_by_score(class_path, get_image_ratio, asc=False)

In [159]:
# removed_filenames are the unwanted images that you manually removed from `dir`
removed_filenames = removing_imgs_prefix(class_path, prefix_to_filename_with_prefix)
removed_filenames

['0000000_fSocialMedia0000995.jpg',
 '0000001_fSocialMedia0002015.jpg',
 '0000002_fSocialMedia0001255.jpg',
 '0000003_fSocialMedia0001438.jpg',
 '0000004_fSocialMedia0001788.jpg',
 '0000005_fSocialMedia0001377.jpg',
 '0000006_fSocialMedia0001379.jpg',
 '0000007_fSocialMedia0001883.jpg',
 '0000008_fSocialMedia0001072.jpg',
 '0000009_fSocialMedia0000242.jpg',
 '0000010_fSocialMedia0001037.jpg',
 '0000011_fSocialMedia0002400.jpg',
 '0000012_fSocialMedia0001376.jpg',
 '0000013_fSocialMedia0001385.jpg',
 '0000014_fSocialMedia0000006.jpg',
 '0000015_fSocialMedia0001499.jpg',
 '0000016_fSocialMedia0001749.jpg',
 '0000017_fSocialMedia0000105.jpg',
 '0000018_fSocialMedia0001382.jpg',
 '0000019_fSocialMedia0001046.jpg',
 '0000020_fSocialMedia0001049.jpg',
 '0000021_fSocialMedia0001313.jpg',
 '0000022_fSocialMedia0001560.jpg',
 '0000023_fSocialMedia0001289.jpg',
 '0000024_fSocialMedia0002418.jpg',
 '0000025_fSocialMedia0001434.jpg',
 '0000026_fSocialMedia0001164.jpg',
 '0000027_fSocialMedia000017

In [161]:
imgs_metadata_v3_ratio_2 = update_metadata_df(imgs_metadata_v3_acdig, removed_filenames)
imgs_metadata_v3_ratio_2[imgs_metadata_v3_ratio_2['class'] == '30. fSocialMedia'].shape

(2544, 37)

Before: 2729 --> after: 2544 (removed 185 images from both very low and very high ratio images)

### By #words

In [162]:
prefix_to_filename_with_prefix = prefix_imgs_by_score(class_path, num_detected_words, asc=False)

In [163]:
# removed_filenames are the unwanted images that you manually removed from `dir`
removed_filenames = removing_imgs_prefix(class_path, prefix_to_filename_with_prefix)
removed_filenames

['0000000_fSocialMedia0002085.jpg',
 '0000001_fSocialMedia0002328.jpg',
 '0000002_fSocialMedia0000113.jpg',
 '0000003_fSocialMedia0002604.jpg',
 '0000004_fSocialMedia0002220.jpg',
 '0000005_fSocialMedia0002033.jpg',
 '0000006_fSocialMedia0002308.jpg',
 '0000007_fSocialMedia0002002.jpg',
 '0000008_fSocialMedia0000129.jpg',
 '0000009_fSocialMedia0000055.jpg',
 '0000010_fSocialMedia0002103.jpg',
 '0000011_fSocialMedia0001997.jpg',
 '0000012_fSocialMedia0002237.jpg',
 '0000013_fSocialMedia0000204.jpg',
 '0000014_fSocialMedia0000637.jpg',
 '0000015_fSocialMedia0000310.jpg',
 '0000016_fSocialMedia0002605.jpg',
 '0000017_fSocialMedia0000565.jpg',
 '0000018_fSocialMedia0002228.jpg',
 '0000019_fSocialMedia0002074.jpg',
 '0000020_fSocialMedia0000670.jpg',
 '0000021_fSocialMedia0000149.jpg',
 '0000022_fSocialMedia0000489.jpg',
 '0000023_fSocialMedia0000883.jpg',
 '0000024_fSocialMedia0001008.jpg',
 '0000025_fSocialMedia0000767.jpg',
 '0000026_fSocialMedia0002258.jpg',
 '0000027_fSocialMedia000229

In [165]:
imgs_metadata_v3_ratio_texts_2 = update_metadata_df(imgs_metadata_v3_ratio_2, removed_filenames)
imgs_metadata_v3_ratio_texts_2[imgs_metadata_v3_ratio_texts_2['class'] == '30. fSocialMedia'].shape

(2094, 37)

Before: 2544 --> after: 2094 (removed 450 images from both very low/high text images)

### By `fastdup` Functions

In [166]:
work_dir = './dataset_related/fastdup_work_dir_fSocialMedia'
input_dir = joinPaths(dataDir, '30. fSocialMedia/')

In [167]:
fd = fastdup.create(work_dir, input_dir)
fd.run()

FastDup Software, (C) copyright 2022 Dr. Amir Alush and Dr. Danny Bickson.

 ########################################################################################

Dataset Analysis Summary: 

    Dataset contains 2094 images
    Valid images are 100.00% (2,094) of the data, invalid are 0.00% (0) of the data
    Similarity:  4.06% (85) belong to 7 similarity clusters (components).
    95.94% (2,009) images do not belong to any similarity cluster.
    Largest cluster has 118 (5.64%) images.
    For a detailed analysis, use `.connected_components()`
(similarity threshold used is 0.9, connected component threshold used is 0.96).

    Outliers: 6.35% (133) of images are possible outliers, and fall in the bottom 5.00% of similarity values.
    For a detailed list of outliers, use `.outliers()`.


In [172]:
fastdup.run_kmeans(input_dir, work_dir, num_clusters=20, nearest_neighbors_k=1, high_accuracy=True)

FastDup Software, (C) copyright 2022 Dr. Amir Alush and Dr. Danny Bickson.


0

In [173]:
fastdup.create_kmeans_clusters_gallery(work_dir, save_path='./dataset_related/fastdup_fSocialMedia_kmeans')

100%|██████████| 16/16 [00:14<00:00,  1.13it/s]


Finished OK. Components are stored as image files ./dataset_related/fastdup_fSocialMedia_kmeans/components_[index].jpg
Stored kmeans clusters visual view in  ./dataset_related/fastdup_fSocialMedia_kmeans\components.html
Execution time in seconds 17.3


0

In [175]:
kmeans_df = pd.read_csv(joinPaths(work_dir, 'kmeans_assignments.csv'))

In [176]:
kmeans_cluster_filenames = kmeans_df.groupby('cluster').agg({'filename': list, 'distance': max})
cluster_to_filenames = {}
for cluster_idx in kmeans_cluster_filenames.index:
    cluster_to_filenames[cluster_idx] = kmeans_cluster_filenames.loc[cluster_idx, 'filename']

total = 0
for cluster_idx, filenames in cluster_to_filenames.items():
    print(cluster_idx, len(filenames))
    total += len(filenames)
print(total)

0 13
1 96
2 2
3 79
4 217
5 201
6 148
7 199
8 43
9 1
10 1
11 1
12 203
13 121
14 35
15 1
16 132
17 268
18 71
19 262
2094


Before: 2094 --> after: 2059

Checking out cluster 14's images from `___/components.html`, 
we see that these images are visually similar to `eGreetingsAndMisc` class, so they'll be removed:

<img src='./project_media/fSocialMedia similar to eGreetingsAndMisc.png' width=1000 />

In [177]:
filepaths_to_remove = cluster_to_filenames[14]
len(filepaths_to_remove)

35

In [178]:
removed_filepaths = remove_imgs(filepaths_to_remove)

In [179]:
imgs_metadata_v3_acdig_fsoc = update_metadata_df(imgs_metadata_v3_ratio_texts_2, removed_filepaths, have_prefix=False, full_path=True)
imgs_metadata_v3_acdig_fsoc[imgs_metadata_v3_acdig_fsoc['class'] == '30. fSocialMedia'].shape

(2059, 37)

## Filtering `ememes` Dataset

### By Ratio

In [187]:
class_path = joinPaths(dataDir, '20. ememes/')

prefix_to_filename_with_prefix = prefix_imgs_by_score(class_path, get_image_ratio, asc=False)

In [188]:
# removed_filenames are the unwanted images that you manually removed from `dir`
removed_filenames = removing_imgs_prefix(class_path, prefix_to_filename_with_prefix)
removed_filenames

['0000000_ememes0011136.jpg',
 '0000001_ememes0010646.jpg',
 '0000002_ememes0011093.jpg',
 '0000003_ememes0009667.jpg',
 '0000004_ememes0005666.jpg',
 '0000005_ememes0005295.jpg',
 '0000006_ememes0005159.jpg',
 '0000007_ememes0010216.jpg',
 '0000008_ememes0011919.jpg',
 '0000009_ememes0002049.jpg',
 '0000010_ememes0001381.jpg',
 '0000011_ememes0006079.jpg',
 '0000012_ememes0010220.jpg',
 '0000013_ememes0007671.jpg',
 '0000014_ememes0005498.jpg',
 '0000015_ememes0005617.jpg',
 '0000016_ememes0010995.jpg',
 '0000017_ememes0001639.jpg',
 '0000018_ememes0000811.jpg',
 '0000019_ememes0010225.jpg',
 '0000020_ememes0009227.jpg',
 '0000021_ememes0010402.jpg',
 '0000022_ememes0009585.jpg',
 '0000023_ememes0004985.jpg',
 '0000024_ememes0004781.jpg',
 '0000025_ememes0001507.jpg',
 '0000026_ememes0009487.jpg',
 '0000027_ememes0006560.jpg',
 '0000028_ememes0004136.jpg',
 '0000029_ememes0008011.jpg',
 '0000030_ememes0005353.jpg',
 '0000031_ememes0006017.jpg',
 '0000032_ememes0007653.jpg',
 '0000033_

In [190]:
imgs_metadata_v3_ratio_3 = update_metadata_df(imgs_metadata_v3_acdig_fsoc, removed_filenames)
imgs_metadata_v3_ratio_3[imgs_metadata_v3_ratio_3['class'] == '20. ememes'].shape

(13133, 37)

Before: 13279 --> after: 13133 (removed 146 images from both very low and very high ratio images)

### By #words

In [191]:
prefix_to_filename_with_prefix = prefix_imgs_by_score(class_path, num_detected_words, asc=False)

In [192]:
# removed_filenames are the unwanted images that you manually removed from `dir`
removed_filenames = removing_imgs_prefix(class_path, prefix_to_filename_with_prefix)
removed_filenames

['0000000_ememes0010842.jpg',
 '0000001_ememes0004782.jpg',
 '0000002_ememes0009528.jpg',
 '0000022_ememes0011676.jpg',
 '0000029_ememes0001301.jpg',
 '0000051_ememes0010605.jpg',
 '0000057_ememes0001356.jpg',
 '0000062_ememes0003268.jpg',
 '0000072_ememes0012645.jpg',
 '0000076_ememes0009229.jpg',
 '0000077_ememes0012118.jpg',
 '0000080_ememes0011606.jpg',
 '0000081_ememes0001269.jpg',
 '0000089_ememes0011649.jpg',
 '0000316_ememes0011011.jpg',
 '0000319_ememes0012749.jpg',
 '0000321_ememes0001428.jpg',
 '0000741_ememes0010794.jpg',
 '0001703_ememes0010292.jpg',
 '0010924_ememes0004772.jpg',
 '0012190_ememes0002083.jpg',
 '0012191_ememes0002159.jpg',
 '0012192_ememes0002160.jpg',
 '0012193_ememes0002295.jpg',
 '0012194_ememes0002568.jpg',
 '0012195_ememes0002620.jpg',
 '0012196_ememes0002831.jpg',
 '0012197_ememes0002841.jpg',
 '0012198_ememes0002842.jpg',
 '0012199_ememes0002843.jpg',
 '0012200_ememes0002844.jpg',
 '0012201_ememes0002859.jpg',
 '0012202_ememes0002904.jpg',
 '0012203_

In [194]:
imgs_metadata_v3_ratio_texts_3 = update_metadata_df(imgs_metadata_v3_ratio_3, removed_filenames)
imgs_metadata_v3_ratio_texts_3[imgs_metadata_v3_ratio_texts_3['class'] == '20. ememes'].shape

(12170, 37)

In [201]:
imgs_metadata_v3_acdig_fsoc_ememes = imgs_metadata_v3_ratio_texts_3.copy()

Before: 13133 --> after: 12170 (removed 963 images from both very low/high text images)

### By `fastdup` Functions

In [195]:
work_dir = './dataset_related/fastdup_work_dir_ememes'
input_dir = joinPaths(dataDir, '20. ememes/')

In [196]:
fd = fastdup.create(work_dir, input_dir)
fd.run()

FastDup Software, (C) copyright 2022 Dr. Amir Alush and Dr. Danny Bickson.

 ########################################################################################

Dataset Analysis Summary: 

    Dataset contains 12170 images
    Valid images are 100.00% (12,170) of the data, invalid are 0.00% (0) of the data
    Similarity:  5.07% (617) belong to 12 similarity clusters (components).
    94.93% (11,553) images do not belong to any similarity cluster.
    Largest cluster has 76 (0.62%) images.
    For a detailed analysis, use `.connected_components()`
(similarity threshold used is 0.9, connected component threshold used is 0.96).

    Outliers: 6.27% (763) of images are possible outliers, and fall in the bottom 5.00% of similarity values.
    For a detailed list of outliers, use `.outliers()`.


In [197]:
fastdup.run_kmeans(input_dir, work_dir, num_clusters=20, nearest_neighbors_k=1, high_accuracy=True)

FastDup Software, (C) copyright 2022 Dr. Amir Alush and Dr. Danny Bickson.


0

In [198]:
fastdup.create_kmeans_clusters_gallery(work_dir, save_path='./dataset_related/fastdup_ememes_kmeans')

100%|██████████| 20/20 [00:13<00:00,  1.53it/s]


Finished OK. Components are stored as image files ./dataset_related/fastdup_ememes_kmeans/components_[index].jpg
Stored kmeans clusters visual view in  ./dataset_related/fastdup_ememes_kmeans\components.html
Execution time in seconds 16.9


0

In [199]:
kmeans_df = pd.read_csv(joinPaths(work_dir, 'kmeans_assignments.csv'))

In [200]:
kmeans_cluster_filenames = kmeans_df.groupby('cluster').agg({'filename': list, 'distance': max})
cluster_to_filenames = {}
for cluster_idx in kmeans_cluster_filenames.index:
    cluster_to_filenames[cluster_idx] = kmeans_cluster_filenames.loc[cluster_idx, 'filename']

total = 0
for cluster_idx, filenames in cluster_to_filenames.items():
    print(cluster_idx, len(filenames))
    total += len(filenames)
print(total)

0 877
1 507
2 761
3 529
4 191
5 683
6 512
7 1009
8 621
9 699
10 285
11 842
12 637
13 475
14 706
15 662
16 374
17 501
18 715
19 584
12170


Due to the high variance nature of the class, the images per cluster were different to a degree which prevents us from removing an entire cluster

## Filtering `academicPhotos` Dataset

### By Ratio

In [202]:
class_path = joinPaths(dataDir, '81. academicPhotos/')

prefix_to_filename_with_prefix = prefix_imgs_by_score(class_path, get_image_ratio, asc=False)

In [203]:
# removed_filenames are the unwanted images that you manually removed from `dir`
removed_filenames = removing_imgs_prefix(class_path, prefix_to_filename_with_prefix)
removed_filenames

['0000000_academicPhotos0002964.jpg',
 '0000001_academicPhotos0003194.jpg',
 '0000002_academicPhotos0004354.jpg',
 '0000003_academicPhotos0003426.jpg',
 '0000004_academicPhotos0004355.jpg',
 '0000005_academicPhotos0003525.jpg',
 '0000006_academicPhotos0004531.jpg',
 '0000007_academicPhotos0002483.jpg',
 '0000008_academicPhotos0002484.jpg',
 '0000009_academicPhotos0002970.jpg',
 '0000010_academicPhotos0003200.jpg',
 '0000011_academicPhotos0003211.jpg',
 '0000012_academicPhotos0005432.jpg',
 '0000013_academicPhotos0000991.jpg',
 '0000014_academicPhotos0002353.jpg',
 '0000015_academicPhotos0002455.jpg',
 '0000016_academicPhotos0003414.jpg',
 '0000017_academicPhotos0002845.jpg',
 '0000018_academicPhotos0003256.jpg',
 '0000019_academicPhotos0002858.jpg',
 '0000020_academicPhotos0002469.jpg',
 '0000021_academicPhotos0000977.jpg',
 '0000022_academicPhotos0001003.jpg',
 '0000023_academicPhotos0003006.jpg',
 '0000024_academicPhotos0002303.jpg',
 '0000025_academicPhotos0001954.jpg',
 '0000026_ac

In [205]:
imgs_metadata_v3_ratio_4 = update_metadata_df(imgs_metadata_v3_acdig_fsoc_ememes, removed_filenames)
imgs_metadata_v3_ratio_4[imgs_metadata_v3_ratio_4['class'] == '81. academicPhotos'].shape

(5722, 37)

Before: 6035 --> after: 5722 (removed 313 images from both very low and very high ratio images)

### By #words

In [206]:
prefix_to_filename_with_prefix = prefix_imgs_by_score(class_path, num_detected_words, asc=False)

In [207]:
# removed_filenames are the unwanted images that you manually removed from `dir`
removed_filenames = removing_imgs_prefix(class_path, prefix_to_filename_with_prefix)
removed_filenames

['0000000_academicPhotos0000464.jpg',
 '0000001_academicPhotos0005939.jpg',
 '0000002_academicPhotos0000042.jpg',
 '0000003_academicPhotos0000670.jpg',
 '0000004_academicPhotos0005934.jpg',
 '0000005_academicPhotos0004477.jpg',
 '0000006_academicPhotos0002694.jpg',
 '0000007_academicPhotos0002948.jpg',
 '0000008_academicPhotos0002693.jpg',
 '0000009_academicPhotos0004480.jpg',
 '0000010_academicPhotos0005804.jpg',
 '0000011_academicPhotos0004386.jpg',
 '0000012_academicPhotos0000677.jpg',
 '0000013_academicPhotos0005812.jpg',
 '0000014_academicPhotos0002681.jpg',
 '0000015_academicPhotos0000146.jpg',
 '0000016_academicPhotos0005858.jpg',
 '0000017_academicPhotos0002618.jpg',
 '0000018_academicPhotos0001240.jpg',
 '0000019_academicPhotos0000675.jpg',
 '0000020_academicPhotos0005033.jpg',
 '0000021_academicPhotos0005098.jpg',
 '0000022_academicPhotos0002582.jpg',
 '0000023_academicPhotos0000801.jpg',
 '0000039_academicPhotos0003452.jpg',
 '0004913_academicPhotos0001410.jpg',
 '0004914_ac

In [209]:
imgs_metadata_v3_ratio_texts_4 = update_metadata_df(imgs_metadata_v3_ratio_4, removed_filenames)
imgs_metadata_v3_ratio_texts_4[imgs_metadata_v3_ratio_texts_4['class'] == '81. academicPhotos'].shape

(5560, 37)

Before: 5722 --> after: 5560 (removed 162 images from both very low/high text images)

### By `fastdup` Functions

In [210]:
work_dir = './dataset_related/fastdup_work_dir_academicPhotos'
input_dir = joinPaths(dataDir, '81. academicPhotos/')

In [211]:
fd = fastdup.create(work_dir, input_dir)
fd.run()

FastDup Software, (C) copyright 2022 Dr. Amir Alush and Dr. Danny Bickson.

 ########################################################################################

Dataset Analysis Summary: 

    Dataset contains 5560 images
    Valid images are 100.00% (5,560) of the data, invalid are 0.00% (0) of the data
    Similarity:  11.17% (621) belong to 17 similarity clusters (components).
    88.83% (4,939) images do not belong to any similarity cluster.
    Largest cluster has 72 (1.29%) images.
    For a detailed analysis, use `.connected_components()`
(similarity threshold used is 0.9, connected component threshold used is 0.96).

    Outliers: 6.31% (351) of images are possible outliers, and fall in the bottom 5.00% of similarity values.
    For a detailed list of outliers, use `.outliers()`.


In [212]:
fastdup.run_kmeans(input_dir, work_dir, num_clusters=10, nearest_neighbors_k=1, high_accuracy=True)

FastDup Software, (C) copyright 2022 Dr. Amir Alush and Dr. Danny Bickson.


0

In [213]:
fastdup.create_kmeans_clusters_gallery(work_dir, save_path='./dataset_related/fastdup_academicPhotos_kmeans')

100%|██████████| 10/10 [00:23<00:00,  2.31s/it]


Finished OK. Components are stored as image files ./dataset_related/fastdup_academicPhotos_kmeans/components_[index].jpg
Stored kmeans clusters visual view in  ./dataset_related/fastdup_academicPhotos_kmeans\components.html
Execution time in seconds 24.8


0

In [214]:
kmeans_df = pd.read_csv(joinPaths(work_dir, 'kmeans_assignments.csv'))

In [215]:
kmeans_cluster_filenames = kmeans_df.groupby('cluster').agg({'filename': list, 'distance': max})
cluster_to_filenames = {}
for cluster_idx in kmeans_cluster_filenames.index:
    cluster_to_filenames[cluster_idx] = kmeans_cluster_filenames.loc[cluster_idx, 'filename']

total = 0
for cluster_idx, filenames in cluster_to_filenames.items():
    print(cluster_idx, len(filenames))
    total += len(filenames)
print(total)

0 474
1 333
2 966
3 653
4 708
5 566
6 18
7 836
8 755
9 251
5560


Before: 5560 --> after: 5542

Checking out cluster 6's images from `___/components.html`, 
we see that these images are not related to any academic documents/websites so we'll consider them outliers and remove them

<img src='./project_media/academicPhotos misc images.png' width=1000 />

In [216]:
filepaths_to_remove = cluster_to_filenames[6]
len(filepaths_to_remove)

18

In [217]:
removed_filepaths = remove_imgs(filepaths_to_remove)

In [218]:
imgs_metadata_v3_acdig_fsoc_ememes_acpho = update_metadata_df(imgs_metadata_v3_ratio_texts_4, removed_filepaths, have_prefix=False, full_path=True)
imgs_metadata_v3_acdig_fsoc_ememes_acpho[imgs_metadata_v3_acdig_fsoc_ememes_acpho['class'] == '81. academicPhotos'].shape

(5542, 37)

In [219]:
imgs_metadata_v3 = imgs_metadata_v3_acdig_fsoc_ememes_acpho.copy()

## Saving New `ImgsProps` & `train/val/test` CSVs

In [224]:
# Saving new ImgsProps
# imgs_metadata_v3.to_csv(dataReDir+'imgsPropsv3.csv', index=False) # uncomment if you run code above with updated images
imgPropertiesDf = pd.read_csv(dataReDir+'imgsPropsv3.csv')

C:\Users\ashra\AppData\Local\Temp\ipykernel_26220\2036813271.py:3: DtypeWarning:

Columns (25,28,30) have mixed types. Specify dtype option on import or set low_memory=False.



In [227]:
imgPropertiesDf.sample(n=2, random_state=42,)

,file_name,relative_path,aspect_ratio,area,width,height,dominant_color_1,color_to_image_ratio_1,dominant_color_2,color_to_image_ratio_2,dominant_color_3,color_to_image_ratio_3,dominant_color_4,color_to_image_ratio_4,dominant_color_5,color_to_image_ratio_5,total_faces_count,faces_to_image_ratio,text_to_image_ratio,lines_per_text_block,total_lines_of_text_count,text_blocks_count,total_en_boxes,total_ar_boxes,en_words_original,ar_words_original,en_words_lemmatized,en_words_segmented_and_lemmatized,ar_words_lemmatized,en_words_including_separators,ar_words_including_separators,en_words_to_boxes,ar_words_to_boxes,en_avg_score,ar_avg_score,math_digits_or_symbols_count,class
125331,eGreetingAndMisc0013127.jpg,dataset/70. eGreetingAndMisc,1.01,50398,226,223,16777215,0.07435,16776959,0.01077,16776703,0.00845,16711679,0.00609,16776191,0.00597,0,0.0,0.12585,1.0,0,0,2,0,Harpy !Ehrthda,NaN,harpy ehrthda,harpy ehr thda,NaN,Harpy !Ehrthda,NaN,"{'Harpy': [[[57.0, 42.0], [174.0, 44.0], [173.0, 72.0], [56.0, 70.0]]], '!Ehrthda': [[[25.0, 69.0], [199.0, 75.0], [198.0, 117.0], [24.0, 111.0]]]}",{},0.72827,0.00000,0,70. eGreetingAndMisc
86344,ememes0009037.jpg,dataset/20. ememes,1.32,393120,720,546,16777215,0.18495,6317937,0.00599,16711422,0.00535,0,0.00521,6450296,0.00440,0,0.0,0.47453,2.0,1,1,5,9,Egyption boys She loves me,بنت بتقول لولد ممكن تدي للسواق واحد هه خمسه,egyption boy love,egyption boy love,بنت تقول ولد ممكن وتد سواق واحد هه خمس,Egyption boys ||| She loves me,بنت بتقول لولد ممكن تدي للسواق واحد هه خمسه,"{'Egyption': [[[25.0, 93.0], [195.0, 97.0], [194.0, 136.0], [24.0, 132.0]]], 'boys': [[[207.0, 93.0], [326.0, 99.0], [324.0, 139.0], [205.0, 133.0]]], 'She': [[[235.0, 440.0], [318.0, 440.0], [318.0, 478.0], [235.0, 478.0]]], 'loves': [[[326.0, 441.0], [441.0, 441.0], [441.0, 477.0], [326.0, 477.0]]], 'me': [[[449.0, 446.0], [515.0, 446.0], [515.0, 479.0], [449.0, 479.0]]]}","{'خمسه': [[[24.0, 35.0], [105.0, 37.0], [104.0, 67.0], [23.0, 64.0]]], 'هه': [[[110.0, 37.0], [159.0, 37.0], [159.0, 70.0], [110.0, 70.0]]], 'واحد': [[[161.0, 32.0], [228.0, 36.0], [225.0, 71.0], [159.0, 67.0]]], 'للسواق': [[[231.0, 35.0], [328.0, 35.0], [328.0, 72.0], [231.0, 72.0]]], 'تدي': [[[331.0, 37.0], [388.0, 35.0], [390.0, 76.0], [333.0, 78.0]]], 'ممكن': [[[392.0, 37.0], [475.0, 33.0], [477.0, 66.0], [394.0, 70.0]]], 'لولد': [[[480.0, 34.0], [541.0, 37.0], [540.0, 72.0], [478.0, 69.0]]], 'بتقول': [[[545.0, 36.0], [629.0, 36.0], [629.0, 72.0], [545.0, 72.0]]], 'بنت': [[[632.0, 36.0], [687.0, 36.0], [687.0, 72.0], [632.0, 72.0]]]}",0.99556,0.98459,0,20. ememes


In [228]:
# Create a dataframe with the file paths and labels of all images in dataDir
def get_all_images_df(dataDir):
    df = pd.DataFrame({'filepath': [], 'label': []})
    for class_name in [cn for cn in os.listdir(dataDir) if not cn.startswith('.')]: # to avoid hidden folders
        class_dir = joinPaths(dataDir, class_name)
        file_names = os.listdir(class_dir)
        file_paths = [joinPaths(class_dir, file_name) for file_name in file_names]
        labels = [class_name] * len(file_names)
        class_data = pd.DataFrame({'filepath': file_paths, 'label': labels})
        df = pd.concat([df, class_data])
    return df

In [229]:
all_images_df = get_all_images_df(dataDir)
len(all_images_df), all_images_df['label'].value_counts()

(132527,
 00. selfies             46834
 10. fmemes              31016
 50. fTxtMssgs           15336
 70. eGreetingAndMisc    13413
 20. ememes              12170
 81. academicPhotos       5542
 40. eSocialMedia         4789
 30. fSocialMedia         2059
 82. academicDigital      1368
 Name: label, dtype: int64)

In [231]:
# previous dataset (v2):
df_v2 = pd.read_csv('./dataset_related/imgsPropsv2.csv')
len(df_v2), df_v2['class'].value_counts()

C:\Users\ashra\AppData\Local\Temp\ipykernel_26220\2857198920.py:2: DtypeWarning:

Columns (25,28,30) have mixed types. Specify dtype option on import or set low_memory=False.



(135410,
 00. selfies             46834
 10. fmemes              31016
 50. fTxtMssgs           15336
 70. eGreetingAndMisc    13413
 20. ememes              13279
 81. academicPhotos       6035
 40. eSocialMedia         4789
 30. fSocialMedia         2729
 82. academicDigital      1979
 Name: class, dtype: int64)

In [232]:
# Split the dataframe into training (70%), validation (15%), and testing (15%) sets
train_data, val_test_data = train_test_split(all_images_df, test_size=0.3, random_state=42, shuffle=True)
val_data, test_data = train_test_split(val_test_data, test_size=0.5, random_state=42, shuffle=True)

# Saving train/val/test CSVs
# train_data.to_csv(dataReDir+'train_datav02.csv', index=False) # uncomment these 3 lines of code if the images are updated
# val_data.to_csv(dataReDir+'val_datav02.csv', index=False) 
# test_data.to_csv(dataReDir+'test_datav02.csv', index=False) 

train_data = pd.read_csv(dataReDir+'train_datav02.csv')
val_data = pd.read_csv(dataReDir+'val_datav02.csv')
test_data = pd.read_csv(dataReDir+'test_datav02.csv')